# Building a Database for Crime Reports

Using the data from `boston.csv`, a complication of crimes that occurred in Boston, we will construct a functional database with role permissions with `readonly` access for data analysts and `readwrite` for data scientists . 

Description of the data set by column:
- `incident_number`: instance identifier of the crime
- `offense_code`: numeric code type of the crime
- `description`: brief of the crime committed
- `date` & `day_of_the_week`: when the crime happened
- `lat` & `long`: location of the crime with latitude and longitude coordinates

## Creating the database and schema

First we will build a database called `crime_db` by connecting to our existing database `dq`.

In [8]:
import psycopg2
#conn.close()
conn = psycopg2.connect("dbname=dq user=dq")
# set autocommit to True to create database
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db;")
conn.autocommit = False
conn.close()

Next we will create a schema named `crimes` for storing the tables of crime data. 

In [9]:
conn = psycopg2.connect("dbname=crime_db user=dq")
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")
conn.commit()

## Obtain the column names and sample first row

By obtaining the column headers and first row of data as variables, we can more easily explore the data and build the table.

In [10]:
import csv
with open("boston.csv") as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

## Analyzing the column values

Identifying the proper datatypes for each column will be essential before building our table. So we shall construct a function to do this work for us.

In [11]:
def get_col_value_set(csv_filename, col_index):
    values = set()
    with open(csv_filename, 'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return values

for i in range(len(col_headers)):
    values = get_col_value_set("boston.csv", i)
    print(col_headers[i], len(values), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


Next let's use our `get_col_value_set()` function to get a set of text elements and determine which is the longest. Since `Wednesday` is known to be the longest of the `day_of_the_week` column, we shall focus our efforts on the `description` column.

In [12]:
# print the column headers to determine index value of description
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [13]:
descriptions = get_col_value_set("boston.csv", 2)
max_len = 0
for d in descriptions:
    max_len = max(max_len, len(d))
print(max_len)

58


Before we move forward, let's take another look at our column headers and the datatypes of the first row:

In [14]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


## Creating the table

Based on our observations from above, we will use the type `INTEGER` for both `incident_number` and `offense_code`. `incident_number` will serve as our primary key.

Since the `description` has at most 58 characters we will use the datatype `VARCHAR(100)` to represent it. This leaves some margins for larger, future values while not being so big that we will waste a lot of memory.

We will create an enumerated datatype named `weekday` for the `day_of_the_week` since there there only seven possible values.

The date is represented as the `DATE` datatype. Finally, for the latitude and longitude `DECIMAL` datatype will be used to capture the precision needed.

In [15]:
# create the enumerated datatype for representing the weekday
cur.execute("""
    CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
""")
# create the table
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week weekday,
        lat decimal,
        long decimal
    );
""")

## Loading crime data into our table

We will use the `copy_expert` method to load the data quickly:

In [16]:
# load the data from boston.csv into the table boston_crimes that is in the crimes schema
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
cur.execute("SELECT * FROM crimes.boston_crimes")
# print the number of rows to ensure that they were loaded
print(len(cur.fetchall()))

298329


## Creating our users groups for data analysts and data scientists

By following the least privilege principle, the first step in doing so is to make sure that there are no privileges inherited from the public group and on the public schema.

In [17]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

conn.commit()

Next let's create our user `readonly` and `readwrite` groups:

In [18]:
# create the readonly group for data analysts
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")

# create the readwrite group for data scientists
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

conn.commit()

Finally, we will create a user for each group. `data_analyst` will belong to the `readonly` group and `data_scientist` to `readwrite`.

In [19]:
# create user data_analyst and add to readonly group
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")

# create user data_scientist and add to readwrite group
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

conn.commit()

## Testing the database setup

We can use SQL queries to check whether the objects have been created and that users and groups have the right privileges. This requires you to know the Postgres internal tables. We can query the `pg_roles table` to inspect privileges related to the database and the `information_schema.table_privileges` table to inspect table privileges.

In the pg_roles table we will check database related privileges and for that we will look at the following columns:

- `rolname`: The name of the user / group that the privilege refers to.
- `rolsuper`: Whether this user / group is a super user. It should be set to `False` on every user / group that we have created.
- `rolcreaterole`: Whether user / group can create users, groups or roles. It should be `False` on every user / group that we have created.
- `rolcreatedb`: Whether user / group can create databases. It should be `False` on every user / group that we have created.
- `rolcanlogin`: Whether user / group can login. It should be `True` on the users and `False` on the groups that we have created.

In the `information_schema.table_privileges` we will check privileges related to SQL queries on tables. We will list the privileges of each group that we have created.

In [20]:
# close the old connection to test with a brand new connection
conn.close()

conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
# check users and groups
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)
conn.commit()
print()
# check privileges
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur:
    print(user)
conn.commit()
conn.close()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)

('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')
